In [1]:
import random
import os
from datetime import datetime
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')
from rdkit import Chem

In [2]:
# bond - tuple (fromatom, toatom, bondorder)  sorted so that toatom > fromatom
# atom - ordered list of atom names, e.g. 'C'
# molecule - tuple of  (atom, bond)

# hetero atoms, valence, and probabilities for them
hetatoms = {}
hetatoms['S']  =  (2,0.237)
hetatoms['Br'] =  (1,0.02)
hetatoms['I']  =  (1,0.05)
hetatoms['Cl'] =  (1,0.154)
hetatoms['F']  =  (1,0.167)
hetatoms['N']  =  (3,0.645)
hetatoms['O']  =  (2,0.744)
hetatoms['P']  =  (3,0.050)
hetatoms['C']  =  (4,0.850)

def chooseHetero():
    """
    choose a hetero replacement based on probabilities in the
    map
    """
    elements = ['S', 'Br', 'I', 'Cl', 'F', 'N', 'O', 'P' ,'C']
    weights =  [0.237, .02, .05, .154, .167, .645, .744, .050, 1.0]
    
    symbol =  random.choices(elements, weights = weights, k = 1)[0]
    (valence, prob) = hetatoms[symbol]

    return (symbol, hetatoms[symbol])

def heteroatoms(molecule):
    """
    randomly assign some carbons to be heteroatoms based on
    map of probabilities
    """
    atoms, bonds = molecule
     
    # walk over atoms in random order, choose an hetero
    for atom in random.sample(range(len(atoms)), len(atoms)):
        
        if atoms[atom] != 'C': # don't re-change
            continue
            
        het = chooseHetero()
        if het is None: continue

        symbol, (val, prob) = het
        current_valence = valence(bonds, atom)
        if current_valence <= val:
            atoms[atom] = symbol
            
    return (atoms, bonds)



In [3]:
def valence(bonds, atnum):
    """
        count the number of bonds to the given atom
    """
    valence = 0
    for (f, t, bondorder) in bonds:
        if f == atnum or t == atnum:
            valence += bondorder
    
    return valence

In [4]:
def sdfile(molecule):
    """
    create a string that is the SDFile (cf. CTFILE Formats) for this molecule
    """
    atoms, bonds = molecule
    bonds.sort()
    
    # one could assign a name
    name = ''
    comment = ''
    
    result = "%-78s\n" % (name)
    userstring = '  MCMolgen'
    datestring = datetime.now().strftime('%m%d%y%H%M')
    result += userstring + datestring + '2D' + '\n'
    result += '%-80s\n' % (comment)
    result += "%3d%3d  0  0  0  0            999 V2000\n" % (len(atoms), len(bonds))
    
    for symbol in atoms:
        result += "%10.4f%10.4f%10.4f %3s               \n" % ( 0.0, 0.0, 0.0, symbol)
    
    for (f, t, bo) in bonds:
        result += "%3d%3d%3d%3d\n" % (f + 1, t + 1, bo, 0)
    
    result += "M  END\n\n$$$$"
    
    return result

In [5]:
def multiplebonds(molecule, nbond, border):
    """
    randomly select eligible bonds to upgrade to the bond order
    """
    
    if nbond == 0:
        return molecule
    
    atoms, bonds = molecule
    adjust = 0
    if border == 3: adjust = -1
    
    # select nbond bonds randomly from all the bonds
    for index in random.sample(range(len(bonds)), nbond):
        
        f,t,bo = bonds[index]
         
        if bo == border: # already has this bondorder
            continue
            
        fv = valence(bonds, f)  # from atom valence
        tv = valence(bonds, t)  # to   atom valence
        
        if  4 - fv  >=  border + adjust \
        and 4 - tv  >=  border + adjust  \
        and bo != border:
            # upgrade bond
            bonds[index] = (f, t, border)

    return (atoms, bonds)



def testbond(bonds, bond):
    """ 
        test that the bond is valid, that it does not
        already exist in any bond order, and that the atoms
        are not equal
    """
    
    bf,bt,o = bond
    
    if (bf == bt):
        return False
    
    for (f,t,o) in bonds:
        if (bf, bt) == (f,t):
            return False
        
    return True

    
def addbond(bonds, bond):
    """
    add the bond to the list of bonds
    """
    if testbond(bonds, bond):    
        bonds.append(bond)
        
    return bonds


def fixbond( bond ):
    """
    sort bond order so that the fromatom > toatom
    """
    a,b,c = bond
    
    if a > b:
        return (b, a, c)
    
    return (a,b,c)



def tracepath(molecule, start, path):
    
    atoms, bonds = molecule
    for (f, t, o) in bonds:
        
        if t == start:
            if not f in path:
                path.append(f)
                return tracepath(molecule, f, path)
        
        elif f == start:
            if not t in path:
                path.append(t)
                return tracepath(molecule, t, path)
    
            
    return (molecule, start, path)
    
    
            
    
def ringbonds(molecule, nrings):
    """
    add nrings random ring closures
    """
    
    if nrings == 0:
        return molecule
    
    atoms, bonds = molecule
    alist = random.sample(range(len(atoms)), 2)
    
    for i in range(nrings):
        alist = random.sample(range(len(atoms)), 2)
        # random start atom
        bondfrom = alist[0]
        bondto   = alist[1]
                
        b = fixbond((bondfrom, bondto, 1))

        if testbond(bonds, b)             \
        and valence(bonds, bondfrom) < 4  \
        and valence(bonds, bondto)   < 4:
            bonds = addbond(bonds, b)

    return(atoms, bonds)



def makearomatic(molecule):
    """
    create a benzene ring
    """
    
    atoms, bonds = molecule
    
    bondto = random.randrange(0,len(atoms)) # pick an atom to attach to
    if valence(bonds, bondto) > 3:
        return molecule
    
    startatom = len(atoms)
    atoms.append('C')
    bond = fixbond( (startatom, bondto, 1) ) 
    if not testbond(bonds, bond):
        print("problem with makearomatic bond")
        print(bond)
        
    bonds = addbond(bonds, bond)

    for i in range(startatom, startatom + 5):
        atoms.append('C')
        bond = (i, i + 1, (i%2) + 1)
        bonds = addbond(bonds, bond)
    
    i += 1
    bond = (startatom, startatom + 5, (i%2) + 1)
    bonds = addbond(bonds, bond)
   
    return (atoms, bonds)



def makecarboxylic(molecule):
    """
    create a carboxylic acid group
    """
    
    atoms, bonds = molecule
    
    bondto = random.randrange(0,len(atoms)) # pick an atom to attach to
    
    if valence(bonds, bondto) > 3:
        return molecule
    
    newatom = len(atoms) # next atom #
    atoms.append('C')
    bond= fixbond(( bondto, newatom, 1))
    bonds = addbond(bonds, bond)
                  
    newatom2 = len(atoms) # next atom #
    atoms.append('O')
    bond= fixbond(( newatom2, newatom, 2))
    bonds = addbond(bonds, bond)                  
                  
    newatom3 = len(atoms) # next atom #
    atoms.append('O')
    bond= fixbond(( newatom, newatom3, 1))
    bonds = addbond(bonds, bond)                            
                  
    return (atoms, bonds)

In [6]:
def smilesprinter(rdkit_mol):
    return Chem.MolToSmiles(rdkit_mol)


def smileswriter(rdkit_mol, fname):
    
    f = open(fname, 'a')

    try:
        smiles = smilesprinter(rdkit_mol)

        line = smiles + '\n'
        f.write(line)

    except Exception as e:
        print(str(e))

        
def sdprinter(rdkit_mol):
    
        Chem.rdDepictor.Compute2DCoords(rdkit_mol)
        sd = Chem.MolToMolBlock(rdkit_mol)

        sd += "\n$$$$"
        return sd
 
          
def sdfilewriter(rdkit_mol, fname):
    
        f = open(fname, 'a')

        sd = sdprinter(rdkit_mol)
        f.write(sd + '\n')

In [7]:
def initmol():
    
    # initialize with first bond and two atoms
    bonds = [(0,1,1)]
    atoms = ['C','C']
    
    return (atoms, bonds)

def makescaffold(mol, natoms):
    """ 
    create a random scaffold of natoms carbon atoms, without rings
    """
    atoms, bonds = mol
    num_atoms = len(atoms)
    
    # build atom-by-atom
    for bondfrom in range(num_atoms, num_atoms + natoms): # this will be the new atom number
        
        while True: # try until a bond is found somewhere
            # randomly select an atom to connect to
            bondto = random.randrange(0,bondfrom)
            bond = fixbond((bondfrom, bondto, 1))
            
            if testbond(bonds, bond)        \
            and valence(bonds, bondto)   < 4:
                atoms.append('C')
                bonds = addbond(bonds, bond)
                break  # found an atom to bond to
    
    return(atoms, bonds)



def gaussrandom(ave, sd):
    """
    gaussian distribution of positive integers centered on 'ave'
    in theory this could take forever to generate a random number > 0
    but in practice it is ok since ave is > 0
    """
    while True:
        result = int(random.gauss(ave, sd))
        if result > 0:
            return result
   

def makem(natoms):
    
        mol = initmol()
        natoms -= 2
        
        if random.random() < 0.25:
            mol = makearomatic(mol)
            natoms -= 6
        
        # natoms-2 additional atoms because the initial molecule has 2 atoms
        mol = makescaffold(mol, natoms)
        mol = ringbonds(mol, gaussrandom(0,3))
        atoms, bonds = mol
        
        ntriplebonds = random.randrange(0, 1)
        mol = multiplebonds(mol, ntriplebonds, 3)

        ndoublebonds = random.randrange(0,len(bonds))
        mol = multiplebonds(mol, ndoublebonds, 2)
        
        if random.random() < 0.10:
            mol = makecarboxylic(mol)
        
        atoms, bonds = mol
        
        mol = heteroatoms(mol)
        
        return mol
        
    
    
def makemol(nmols, writer, fname):
    
    hash = {}

    # delete output file if exists 
    try:
        if os.path.exists(fname):
            os.remove(fname)
    except:
        pass
            
    for i in range(nmols):
        
        # select size of molecules
        natoms = random.randrange(12, 50)
        mol = makem(natoms)
        
        sd = sdfile(mol)
        rdkit_mol    = Chem.MolFromMolBlock(sd)
        smiles = Chem.MolToSmiles(rdkit_mol)

        try:
            inchi = Chem.inchi.MolToInchi(rdkit_mol, logLevel= None , treatWarningAsError=False)
        except:
            # ignore errors
            continue
        
        # keep track of molecules already generated
        if inchi in hash:
            hash[inchi] += 1
        else:
            hash[inchi] = 1
            writer(rdkit_mol, fname)
         
        # periodic update
        if i % 10000 == 0:
            
            minimum = 100000
            maximum = -1
            
            for i in hash:
                item = hash.get(i)
                if item < minimum:
                    minimum = item
                if item > maximum:
                    maximum = item
             
            # in theory if all molecules are found N many times, the chance that
            # there are others not yet found is 1 in 2^N
            # this can be used for limited size molecules to generate complete sets
            if minimum > 6:
                break
                
            #print('hash size', len(hash), minimum, maximum)
            
    print('hash size', len(hash), minimum, maximum)        
   
  
def findmol(drug):
    """
    find a specific drug by limiting the generation to molecules with the
    correct number of atoms
    """
    seed = 19
    random.seed(seed) # try to make it repeatable
    count = 0
    rdkit_drug = Chem.MolFromSmiles(drug)
    drug = Chem.MolToSmiles(rdkit_drug, canonical = True)
    nheavy = rdkit_drug.GetNumAtoms()
    print(drug, 'atoms', nheavy, 'seed', seed)
    
    while True:
        count += 1
        if count % 1000000 == 0:
            print('count ', count )
            
        mol = makem(nheavy)
        sd = sdfile(mol)
        
        rdkit_mol = Chem.MolFromMolBlock(sd)
        smiles = Chem.MolToSmiles(rdkit_mol, canonical = True)
        
        if smiles == drug:
            print("found", drug, ' attempt', count)
            break
        
 

In [8]:
findmol('CC(C)c1cccc(c1O)C(C)C')

CC(C)c1cccc(C(C)C)c1O atoms 13 seed 19
count  1000000
count  2000000
count  3000000
found CC(C)c1cccc(C(C)C)c1O  attempt 3805355
